<a href="https://colab.research.google.com/github/syedmehrabzaidi/AI-LAB/blob/main/calendar_booking_via_openai_funcalling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# 1) Install / upgrade dependencies
!pip install --quiet --upgrade \
    openai \
    google-api-python-client \
    google-auth-httplib2 \
    google-auth-oauthlib \
    python-dateutil

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.2/13.2 MB 23.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 229.9/229.9 kB 7.7 MB/s eta 0:00:00


In [ ]:
# 2) Upload your OAuth2 Desktop client_secret JSON
from google.colab import files
import json, os
from google_auth_oauthlib.flow import Flow
from googleapiclient.discovery import build


print("Upload your OAuth2 Desktop client_secret_*.json file:")
uploaded = files.upload()
client_secret_file = next(iter(uploaded))

SCOPES = ['https://www.googleapis.com/auth/calendar']
flow = Flow.from_client_secrets_file(
    client_secret_file,
    scopes=SCOPES,
    # redirect_uri='https://webhook.site/84652d45-bf6c-4eb2-a5f5-47cf8263880a'
    redirect_uri='https://webhook.site/5761bb92-10ce-4a4f-bba3-b841912d9257'
)

auth_url, state = flow.authorization_url(
    access_type='offline',
    include_granted_scopes='true',
    prompt='consent'
)
print("Visit this URL and approve access:\n", auth_url)


Upload your OAuth2 Desktop client_secret_*.json file:


Saving client_secret_2.json to client_secret_2.json
Visit this URL and approve access:
 https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=107126035690-doal2v7uh1adkcuk8fs9gejmhh6tn286.apps.googleusercontent.com&redirect_uri=https%3A%2F%2Fwebhook.site%2F5761bb92-10ce-4a4f-bba3-b841912d9257&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcalendar&state=QpucMZJS2CAAcXwo2hVhzDEXIAPW11&access_type=offline&include_granted_scopes=true&prompt=consent


In [ ]:
auth_code = ""
flow.fetch_token(code=auth_code)
creds = flow.credentials
# save for next time
with open('token.json', 'w') as f:
    f.write(creds.to_json())
TOKEN_FILE = 'token.json'

print("✅ Credentials ready; token saved to", TOKEN_FILE)

# 4) Build the Google Calendar API client
from googleapiclient.discovery import build
calendar_service = build('calendar', 'v3', credentials=creds)

NameError: name 'flow' is not defined

In [ ]:
from google_auth_oauthlib.flow import InstalledAppFlow

SCOPES = ['https://www.googleapis.com/auth/calendar.readonly']

# This will open a browser window for user login and automatically fetch the auth_code
flow = InstalledAppFlow.from_client_secrets_file('./client_secret_2.json', SCOPES)
creds = flow.run_local_server(port=0)

# Save the credentials for next time
with open('token.json', 'w') as f:
    f.write(creds.to_json())

print("✅ Credentials ready; token saved to", 'token.json')


Error: could not locate runnable browser

In [ ]:
# 5) Imports, timezone constants, and helpers
from datetime import datetime, timedelta, timezone
from dateutil import parser

# Always use GMT+5 / Pakistan Standard Time
LOCAL_TZ = timezone(timedelta(hours=5))
TIMEZONE = 'Asia/Karachi'

def parse_datetime(dt_str: str) -> datetime:
    """
    Parse a datetime string.
    - If it contains 'T', treat it as full RFC3339.
    - Otherwise, treat as time today in LOCAL_TZ.
    """
    if 'T' in dt_str:
        return parser.isoparse(dt_str)
    # parse only time (with or without AM/PM)
    t = parser.parse(dt_str)
    now = datetime.now(LOCAL_TZ)
    return datetime(
        now.year, now.month, now.day,
        t.hour, t.minute, t.second,
        tzinfo=LOCAL_TZ
    )

def invert_busy(busy_slots, start: datetime, end: datetime):
    """Given sorted busy slots, return free intervals between start/end."""
    free = []
    current = start
    for b_start, b_end in sorted(busy_slots):
        if b_start > current:
            free.append((current, b_start))
        current = max(current, b_end)
    if current < end:
        free.append((current, end))
    return free

def intersect_intervals(a, b):
    """Return intersection of two sorted interval lists."""
    i = j = 0
    result = []
    while i < len(a) and j < len(b):
        s1, e1 = a[i]
        s2, e2 = b[j]
        start = max(s1, s2)
        end   = min(e1, e2)
        if start < end:
            result.append((start, end))
        if e1 < e2:
            i += 1
        else:
            j += 1
    return result

# 6) Core helpers that always default to LOCAL_TZ and today if no date given
def get_availability(emails, start: str, end: str):
    start_dt = parse_datetime(start)
    end_dt   = parse_datetime(end)
    body = {
        "timeMin":  start_dt.isoformat(),
        "timeMax":  end_dt.isoformat(),
        "timeZone": TIMEZONE,
        "items":    [{"id": e} for e in emails]
    }
    return calendar_service.freebusy().query(body=body).execute().get('calendars', {})

def schedule_meeting(emails, start: str, end: str, summary="Meeting", description=""):
    start_dt = parse_datetime(start)
    end_dt   = parse_datetime(end)
    event = {
        'summary':     summary,
        'description': description,
        'start':       {'dateTime': start_dt.isoformat(), 'timeZone': TIMEZONE},
        'end':         {'dateTime': end_dt.isoformat(),   'timeZone': TIMEZONE},
        'attendees':   [{'email': e} for e in emails],
        'reminders':   {'useDefault': True},
    }
    return calendar_service.events().insert(
        calendarId='primary',
        body=event,
        sendUpdates='all'
    ).execute()

def find_earliest_availability(emails, search_days=7):
    now       = datetime.now(LOCAL_TZ)
    window_end= now + timedelta(days=search_days)
    fb_body   = {
        "timeMin": now.isoformat(),
        "timeMax": window_end.isoformat(),
        "timeZone": TIMEZONE,
        "items": [{"id": e} for e in emails]
    }
    fb = calendar_service.freebusy().query(body=fb_body).execute().get('calendars', {})

    free_per = []
    for e in emails:
        busy = fb.get(e, {}).get('busy', [])
        slots = [(parser.isoparse(x['start']), parser.isoparse(x['end'])) for x in busy]
        free_per.append(invert_busy(slots, now, window_end))

    common = free_per[0]
    for slots in free_per[1:]:
        common = intersect_intervals(common, slots)

    if not common:
        return {"error": f"No common free time in next {search_days} days."}
    s, e = common[0]
    return {"earliest_start": s.isoformat(), "earliest_end": e.isoformat()}

def find_and_schedule_meeting(emails, duration, summary="Meeting", description="", search_days=7):
    avail = find_earliest_availability(emails, search_days)
    if "error" in avail:
        return avail
    start_dt = parser.isoparse(avail["earliest_start"])
    end_dt   = start_dt + timedelta(minutes=duration)
    evt = schedule_meeting(
        emails,
        start_dt.isoformat(),
        end_dt.isoformat(),
        summary, description
    )
    return {
        "scheduled_start": start_dt.isoformat(),
        "scheduled_end":   end_dt.isoformat(),
        "event":           evt
    }

# 7) Instantiate the OpenAI client (v1.x style)
from openai import OpenAI
openai_api_key = ""
client = OpenAI(api_key=openai_api_key)

# 8) Define function schemas for ChatGPT
function_defs = [
    {
        "name":        "get_availability",
        "description": "Check colleagues' free/busy status",
        "parameters":  {
            "type": "object",
            "properties": {
                "emails": {"type":"array","items":{"type":"string"}},
                "start":  {"type":"string"},
                "end":    {"type":"string"}
            },
            "required": ["emails","start","end"]
        },
    },
    {
        "name":        "find_earliest_availability",
        "description": "Find the earliest common free time among participants",
        "parameters":  {
            "type": "object",
            "properties": {
                "emails":      {"type":"array","items":{"type":"string"}},
                "search_days": {"type":"integer", "description":"how many days ahead to search"}
            },
            "required": ["emails"]
        },
    },
    {
        "name":        "schedule_meeting",
        "description": "Schedule a meeting at a specified time",
        "parameters":  {
            "type": "object",
            "properties": {
                "emails":      {"type":"array","items":{"type":"string"}},
                "start":       {"type":"string"},
                "end":         {"type":"string"},
                "summary":     {"type":"string"},
                "description": {"type":"string"}
            },
            "required": ["emails","start","end"]
        },
    },
    {
        "name":        "find_and_schedule_meeting",
        "description": "Find earliest slot for a duration and book it",
        "parameters":  {
            "type": "object",
            "properties": {
                "emails":      {"type":"array","items":{"type":"string"}},
                "duration":    {"type":"integer", "description":"minutes"},
                "summary":     {"type":"string"},
                "description": {"type":"string"},
                "search_days": {"type":"integer","description":"how many days ahead to search"}
            },
            "required": ["emails","duration"]
        },
    },
]

# 9) Chat loop with function-calling
import json

messages = [{
    "role": "system",
    "content": "You can check availability, find earliest free times, or schedule meetings—all times default to GMT+5 and today if date not given."
}]

print("\nChat now! (type 'exit' or 'quit' to stop)\n")
while True:
    user_input = input("You: ")
    if user_input.lower() in ("exit", "quit"):
        break
    messages.append({"role":"user","content":user_input})

    resp = client.chat.completions.create(
        model="gpt-4-0613",
        messages=messages,
        functions=function_defs,
        function_call="auto",
    )
    msg = resp.choices[0].message

    if msg.function_call:
        name = msg.function_call.name
        args = json.loads(msg.function_call.arguments)

        if name == "get_availability":
            result = get_availability(**args)
        elif name == "find_earliest_availability":
            result = find_earliest_availability(**args)
        elif name == "schedule_meeting":
            result = schedule_meeting(**args)
        else:  # find_and_schedule_meeting
            result = find_and_schedule_meeting(**args)

        messages.append({
            "role":    "function",
            "name":    name,
            "content": json.dumps(result, indent=2)
        })
        followup = client.chat.completions.create(
            model="gpt-4-0613",
            messages=messages,
        )
        assistant_reply = followup.choices[0].message.content
    else:
        assistant_reply = msg.content

    messages.append({"role":"assistant","content":assistant_reply})
    print("Bot:", assistant_reply)



Chat now! (type 'exit' or 'quit' to stop)



KeyboardInterrupt: Interrupted by user